### Respiratoy sound automatic annotation (1D Conv, without spectrograms, no downsampling)
Attempt automatic annotation of sound files in the same format as the provided text files.
Use the sound data directly without creating spectograms

In [ ]:
import os
import glob
import copy
import re
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import soundfile as sf

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, precision_recall_curve, plot_precision_recall_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow import keras

try:
    os.environ['KAGGLE_DATA_PROXY_TOKEN']
except KeyError:
    dir_out = "./"
    dir_files = "Respiratory_Sound_Database/Respiratory_Sound_Database/"
    dir_annot_csv = "./"
    fname_demo = dir_path + "demographic_info.txt"
else:
    dir_out = "/kaggle/working/"
    dir_files = "/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/"
    dir_annot_csv = "/kaggle/input/recording-annotations/"
    fname_demo = "/kaggle/input/respiratory-sound-database/" + "demographic_info.txt"
    
fname_diag = dir_files + "patient_diagnosis.csv"
fname_annot = dir_annot_csv + "rec_annotation.csv"
tfrecord_wavs = dir_out + "wavs.tfrecord"
tfrecord_annot = dir_out + "annot.tfrecord"
dir_audio = dir_files + "audio_and_txt_files/"

In [ ]:
group_pat_num = "([0-9]{3})"
group_rec_index = "([0-9][a-z][0-9])"
group_chest_loc = "(Tc|Al|Ar|Pl|Pr|Ll|Lr)"
group_acc_modes = "(sc|mc)"
group_equipments = "(AKGC417L|LittC2SE|Litt3200|Meditron)"

regex_info = re.compile("_".join([group_pat_num, group_rec_index, group_chest_loc, group_acc_modes, group_equipments]))

top = os.getcwd()
os.chdir(dir_audio)
fnames = glob.glob("*.wav")

l_wav_rec = []
dict_wav_rec = {}
min_len = np.inf
max_len = 0

for fname in fnames:
    match_info = regex_info.match(fname)
    pat_num = int(match_info.group(1))
    rec_index = match_info.group(2)
    chest_loc = match_info.group(3)
    acc_mode = match_info.group(4)
    equipment = match_info.group(5)
    
    wav_content = sf.read(fname)[0]
    l_wav_rec.append([pat_num, rec_index, chest_loc, wav_content])
    dict_wav_rec[(pat_num, rec_index, chest_loc)] = wav_content
    
    if len(wav_content) > max_len:
        max_len = len(wav_content)
        # for getting the corresponding annotation below
        max_patnum = pat_num
        max_recindex = rec_index
        max_chestloc = chest_loc
    
    if len(wav_content) < min_len:
        min_len = len(wav_content)
        # for getting the corresponding annotation below
        min_patnum = pat_num
        min_recindex = rec_index
        min_chestloc = chest_loc

os.chdir(top)

# pad all recordings to same length
for i in range(len(l_wav_rec)):
    if len(l_wav_rec[i][3]) < max_len:
        padding = [0] * ( max_len - len(l_wav_rec[i][3]) )
        l_wav_rec[i][3] = np.append(l_wav_rec[i][3], padding)

# pad all recordings to multiple of length of shortest recording
# for i in range(len(l_wav_rec)):
#     if len(l_wav_rec[i][3]) % min_len != 0:
#         padding = [0] * ( min_len - len(l_wav_rec[i][3]) % min_len)
#         l_wav_rec[i][3] = np.append(l_wav_rec[i][3], padding)

l_wav_rec.sort(key=lambda subl: (subl[0], subl[1], subl[2]))
wav_cols = ["Patient number", "Recording index", "Chest location", "WAV"]
df_wav_rec = pd.DataFrame(l_wav_rec, columns=wav_cols)

In [ ]:
# get annotation corresponding to smallest wave file
df_annot = pd.read_csv(fname_annot).set_index(["Patient number", "Recording index", "Chest location"])
df_annot = df_annot.sort_index()
# df_min_annot = df_annot.loc[min_patnum, min_recindex, min_chestloc]
# df_max_annot = df_annot.loc[max_patnum, max_recindex, max_chestloc]

max_len_annot = 0
max_ix = None

for ix in df_annot.index:
    cur_annot = df_annot.loc[ix, ["Cycle start", "Cycle end", "Crackles", "Wheezes"]]
    # pad additional "dummy" rows to maximum length annotation
    if len(cur_annot) > max_len_annot:
        max_len_annot = len(cur_annot)
        df_max_annot = cur_annot
#         max_ix = ix

# construct CNN target feature maps
target_feature_maps = np.empty((0,max_len_annot,4), dtype="float32")
for ix in df_annot.index:
    cur_annot = df_annot.loc[ix, ["Cycle start", "Cycle end", "Crackles", "Wheezes"]].to_numpy()
    # pad additional "dummy" rows to maximum length annotation
    if len(cur_annot) < max_len_annot:
        pad_len = max_len_annot - len(cur_annot)
        # what's the best way of padding?
        padding = np.array([[-1, -1, 0, 0]] * pad_len)
        cur_annot = np.vstack((cur_annot, padding))
    feature_map = cur_annot[np.newaxis, :]
    target_feature_maps = np.append(target_feature_maps, feature_map, axis=0)

In [ ]:
# for val_ix in range(len(l_wav_rec[0][3])):
#     feature = Feature(
#         float_list = FloatList(value=([l_wav_rec[0][3][val_ix]]))
#     )
    
#     feats = [feature]
    
#     feat_list = FeatureList(feature = feats)
    
#     feat_lists_dict["{} {} {} {}".format(l_wav_rec[0][0], l_wav_rec[0][1], l_wav_rec[0][2], val_ix)] = feat_list
    
#     sequence_description["{} {} {} {}".format(l_wav_rec[0][0], l_wav_rec[0][1], l_wav_rec[0][2], val_ix)] = tf.io.FixedLenSequenceFeature([1], tf.float32)

# tfr_options = tf.io.TFRecordOptions(compression_type="GZIP")
tfr_options = tf.io.TFRecordOptions()
f = tf.io.TFRecordWriter(tfrecord_wavs, tfr_options)

for rec in l_wav_rec:
    
#     feat_patnum = Feature(
#         int64_list = Int64List(value=[rec[0]])
#     )

#     feat_recix = Feature(
#         bytes_list = BytesList(value=[bytes(rec[1], "ascii")])
#     )

#     feat_chestloc = Feature(
#         bytes_list = BytesList(value=[bytes(rec[2], "ascii")])
#     )

#     features_context = Features(
#         feature = {
#             "patient number" : feat_patnum,
#             "recording index" : feat_recix,
#             "chest location" : feat_chestloc
#         }
#     )
    
    
    
#     feat_list = [ Feature(
#             float_list = FloatList(value=(rec[3]))
#         )
#     ]


#     feat_lists_dict = {}    
#     feat_lists_dict["WAV"] = FeatureList(feature = feat_list)
#     feat_lists = FeatureLists(feature_list = feat_lists_dict)
    
#     feat_rec_str = Feature(
#         bytes_list = BytesList(value=[bytes("{}_{}_{}".format(rec[0], rec[1], rec[2]), "ascii")])
#     )
#     features_context = Features(
#         feature = {
#             "Recording string" : feat_rec_str
#         }
#     )

    

#     se = SequenceExample(
#         context = features_context,
#         feature_lists = feat_lists
#     )


#     f.write(se.SerializeToString())
    
    
    reshaped_tensor = tf.reshape(tf.constant(rec[3], dtype=tf.float32), shape=[-1, 1])
    f.write(tf.io.serialize_tensor(reshaped_tensor).numpy())
    

f.close()

# free RAM
del l_wav_rec



with tf.io.TFRecordWriter(tfrecord_annot, tfr_options) as f:
    f.write(tf.io.serialize_tensor(tf.constant(target_feature_maps, dtype=tf.float32)).numpy())

# del target_feature_maps
    
# feature_context = {
#     "patient number" : tf.io.FixedLenFeature([], tf.int64),
#     "recording index" : tf.io.FixedLenFeature([], tf.string),
#     "chest location" : tf.io.FixedLenFeature([], tf.string)
#     }

# feature_context = {
#     "Recording string" : tf.io.FixedLenFeature([], tf.string)
# }


# sequence_description = {
#     "WAV" : tf.io.FixedLenSequenceFeature([max_len], tf.float32)
# }

# for serialized in tf.data.TFRecordDataset([fname_tfrecord]).batch(10):
#     parsed = tf.io.parse_sequence_example(serialized, feature_context, sequence_description)
    
# for serialized in tf.data.TFRecordDataset([fname_tfrecord], compression_type="GZIP").batch(10):
#     for s in serialized:
#         parsed_tensor = tf.io.parse_tensor(s, tf.float32)
#         print(parsed_tensor)


    

# print(tf.reshape(tf.constant(parsed[1]['WAV'], dtype=tf.float32), shape=[-1, 1]))
# print(parsed)
# print(l_wav_rec[2][3])


In [ ]:
# for serialized in tf.data.TFRecordDataset([tfrecord_annot]).batch(2):
#     for s in serialized:
#         parsed_tensor = tf.io.parse_tensor(s, tf.float32)
#         print(parsed_tensor)

In [ ]:
def datasetFromTFRecord(fpath=tfrecord_wavs, batch_size=10):
    
    tfr_wav = tf.data.TFRecordDataset([tfrecord_wavs])
    tfr_annot = tf.data.TFRecordDataset([tfrecord_annot])
#     serialized = tfr_dataset.batch(batch_size)
    parsed_wavs = tfr_wav.map(lambda ser : tf.io.parse_tensor(ser, tf.float32))
    parsed_annot = tfr_annot.map(lambda ser : tf.io.parse_tensor(ser, tf.float32)).unbatch()
#     batch_target = copy.deepcopy(target_feature_maps[:batch_size])
#     batch_target = copy.deepcopy(target_feature_maps[:3])
#     target_data = tf.data.Dataset.from_tensors(batch_target)
#     target_data = tf.expand_dims(tf.constant(batch_target), 0)

    
#     wavs_batch = tf.constant(list(parsed_wavs.batch(batch_size).prefetch(1).as_numpy_iterator()))
    
    zipped = tf.data.Dataset.zip((parsed_wavs, parsed_annot))
    
#     data = tf.data.Dataset.from_tensor_slices((parsed_wavs, parsed_annot))
    
#     np.delete(target_feature_maps, batch_size)
    
#     data = tf.data.Dataset.from_tensor_slices((wavs_batch, target_data))
    
#     return parsed_wavs.batch(batch_size).prefetch(1), target_data.batch(batch_size).prefetch(1)
#     return data.batch(batch_size).prefetch(1)
    
    return zipped.batch(batch_size).prefetch(1)
#     return data.batch(batch_size).prefetch(1)




# for serialized in tf.data.TFRecordDataset([fname_tfrecord], compression_type="GZIP").batch(10):
# for serialized in tf.data.TFRecordDataset([tfrecord_annot]).batch(2):
#     for s in serialized:
#         parsed_tensor = tf.io.parse_tensor(s, tf.float32)
#         print(parsed_tensor)

In [ ]:
# x1 = np.array([[df_wav_rec.loc[0, "WAV"].astype("float32")]])
x1 = np.array([df_wav_rec.loc[0, "WAV"].astype("float32")]).reshape(-1,1)[np.newaxis,:,:]
conv = keras.layers.Conv1D(filters=4,kernel_size=5, strides=2, padding="same", activation="relu")

cnn_strides = [1,2,4]
fmap_size = max_len

model = keras.models.Sequential(name="1DCNN")
for i in range(len(cnn_strides)):
    if i == 0:
        layer_name = "conv_input"
    else:
        layer_name = "conv_{}".format(i)
    model.add(keras.layers.Conv1D(filters = 64 / (2 ** i) , kernel_size= 5 * (2 ** i), strides=cnn_strides[i], padding="same", activation="relu", name=layer_name))
    fmap_size //= cnn_strides[i]

# calculate this in order to get the last stride number that results in a output feature map of
# size of longest rec. annotation
last_strides = fmap_size // max_len_annot + 1

model.add(keras.layers.Conv1D(filters = 4 , kernel_size= 5 * (2 ** (i+1)), strides=last_strides, padding="same", activation="relu", name="conv_output"))

model.compile(loss="mse", metrics=["accuracy"], optimizer="sgd")


# conv(tf.expand_dims(parsed_tensor, 0))
# it = iter(tf.data.TFRecordDataset([fname_tfrecord]).batch(1))
# serialized = next(it).numpy()
# parsed_tensor = tf.io.parse_tensor(serialized[0], tf.float32)
# model.fit(tf.expand_dims(parsed_tensor, 0), target_feature_maps[0][np.newaxis, :, :])

train_set = datasetFromTFRecord()
# model.fit(train_set)
# model.build(input_shape=[None, max_len, 1])
# model.summary()

for e in train_set.batch(1).as_numpy_iterator():
    model.fit(e[0], e[1])
#     out = conv(e[0])
#     print(conv(out))

    

# list(train_set.as_numpy_iterator())

    
# model.train_on_batch()
# target_feature_maps[0][np.newaxis, :, :].shape

# model.fit(l_wav_rec, target_feature_maps)
# model.fit(x2, y)